<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import numpy as np
import pandas as pd
import re
import spacy
from spacy.lang.en import stop_words

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline

import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [20]:
data = pd.read_csv('/datasets/toxic_comments.csv')

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [22]:
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [23]:
print('Соотношение класса 1 признака Toxic - {:.2%}'.format(sum(data['toxic']) / len(data)))

Соотношение класса 1 признака Toxic - 10.16%


Сначала надо очистить текст от ненужных символов, а также лемматизировать его

In [38]:
wnl = WordNetLemmatizer()

In [39]:
corpus = data['text'].values

In [40]:
def text_cleaner(text):
    text = re.sub(r'[^a-zA-Z ]', ' ', text)
    return " ".join(text.split())

In [41]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def text_lemmitezer(text):
    tokens = nltk.word_tokenize(text)
    return ' '.join([wnl.lemmatize(w, get_wordnet_pos(w)) for w in tokens])

In [46]:
#print(nltk.pos_tag(['describes']))

[('describes', 'NNS')]


In [47]:
corpus_lemmatized = []
for i in range(len(corpus)): 
    index = text_lemmitezer(text_cleaner(corpus[i]))
    corpus_lemmatized.append(index)

In [50]:
stopwords = set(nltk_stopwords.words('english'))

Далее разделим корпус на тренировочный и тестовый

In [51]:
X_train, X_test, y_train, y_test = train_test_split(corpus_lemmatized, data['toxic'], test_size=0.2, random_state=42)

Далее преобразуем корпус с помощью TF-IDF векторизации

In [52]:
ti = TfidfVectorizer(stop_words=stopwords)

In [53]:
X_train_ti = ti.fit_transform(X_train)
X_test_ti = ti.transform(X_test)

## Обучение

Обучим модели с помощью 2 разных алгоритмов LogisticRegression и SVC, а также произведем подбор их гиперпараметров.

In [57]:
pipe_lr = Pipeline([('ti', TfidfVectorizer(stop_words=stopwords)),
                 ('model', LogisticRegression())])

parameters = {'model__C': [9, 10, 11],
              'model__max_iter': [1000]}
#lrm = LogisticRegression()
grid_lrm = GridSearchCV(pipe_lr, parameters,
                  cv=5,
                  scoring='f1',
                  n_jobs=-1,
                  verbose=2)
grid_lrm.fit(X_train, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV] END ...................model__C=9, model__max_iter=1000; total time= 1.6min
[CV] END ...................model__C=9, model__max_iter=1000; total time= 1.9min
[CV] END ...................model__C=9, model__max_iter=1000; total time= 1.7min
[CV] END ...................model__C=9, model__max_iter=1000; total time= 1.8min
[CV] END ...................model__C=9, model__max_iter=1000; total time= 1.6min
[CV] END ..................model__C=10, model__max_iter=1000; total time= 1.7min
[CV] END ..................model__C=10, model__max_iter=1000; total time= 1.6min
[CV] END ..................model__C=10, model__max_iter=1000; total time= 1.7min
[CV] END ..................model__C=10, model__max_iter=1000; total time= 1.1min
[CV] END ..................model__C=10, model__max_iter=1000; total time= 1.9min
[CV] END ..................model__C=11, model__max_iter=1000; total time= 2.0min
[CV] END ..................model__C=11, model__ma

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('ti',
                                        TfidfVectorizer(stop_words={'a',
                                                                    'about',
                                                                    'above',
                                                                    'after',
                                                                    'again',
                                                                    'against',
                                                                    'ain',
                                                                    'all', 'am',
                                                                    'an', 'and',
                                                                    'any',
                                                                    'are',
                                                                    'aren',
            

In [58]:
print('Параметры лучшей модели - {}, метрика F1 - {}'.format(grid_lrm.best_score_, grid_lrm.best_params_))

Параметры лучшей модели - 0.7698672713382555, метрика F1 - {'model__C': 11, 'model__max_iter': 1000}


In [59]:
pipe_svc = Pipeline([('ti', TfidfVectorizer(stop_words=stopwords)),
                 ('model', LinearSVC())])

parameters = {'model__C': [0.01, 0.1, 10, 11],
              'model__max_iter': [1000]}
#svc = LinearSVC()
grid_svc = GridSearchCV(pipe_svc, parameters,
                  cv=5,
                  scoring='f1',
                  n_jobs=-1,
                  verbose=2)
grid_svc.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] END ................model__C=0.01, model__max_iter=1000; total time=   6.3s
[CV] END ................model__C=0.01, model__max_iter=1000; total time=   6.0s
[CV] END ................model__C=0.01, model__max_iter=1000; total time=   6.1s
[CV] END ................model__C=0.01, model__max_iter=1000; total time=   6.2s
[CV] END ................model__C=0.01, model__max_iter=1000; total time=   6.4s
[CV] END .................model__C=0.1, model__max_iter=1000; total time=   6.1s
[CV] END .................model__C=0.1, model__max_iter=1000; total time=   6.2s
[CV] END .................model__C=0.1, model__max_iter=1000; total time=   6.2s
[CV] END .................model__C=0.1, model__max_iter=1000; total time=   6.0s
[CV] END .................model__C=0.1, model__max_iter=1000; total time=   6.2s
[CV] END ..................model__C=10, model__max_iter=1000; total time=   9.0s
[CV] END ..................model__C=10, model__ma

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('ti',
                                        TfidfVectorizer(stop_words={'a',
                                                                    'about',
                                                                    'above',
                                                                    'after',
                                                                    'again',
                                                                    'against',
                                                                    'ain',
                                                                    'all', 'am',
                                                                    'an', 'and',
                                                                    'any',
                                                                    'are',
                                                                    'aren',
            

In [60]:
print('Параметры лучшей модели - {}, метрика F1 - {}'.format(grid_svc.best_score_, grid_svc.best_params_))

Параметры лучшей модели - 0.7421427743995489, метрика F1 - {'model__C': 10, 'model__max_iter': 1000}


In [62]:
lrm = LogisticRegression(C = 11)
lrm.fit(X_train_ti, y_train)

/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=11)

In [63]:
print('Метрика F1 на тестовой выборке - {}'.format(f1_score(lrm.predict(X_test_ti), y_test)))

Метрика F1 на тестовой выборке - 0.7685282753775906


## Выводы

В этом проекте была разработана модель классификации токсичности твитов. В ходе работ был очищен текст от не нужных символов, проведена лемматизация текстов. Тексты также очищены от стоп-слов.

Был проведен подбор гиперпараметров для алгоритмов LogisticRegression и SVC. Метрики F1 этих моделей составили 0.76 и 0.74, соответственно. 

В качестве финальной модели была выбрана модель с алгоритмом LogisticRegression, которая на тестовой выборке твитов, показала метрику F1 - 0.76